Importing Libraries and Mounting Gdrive to store collected and processed datasets

In [2]:
import requests
import requests_oauthlib
import socket
import sys
import json
from socket import SHUT_RDWR
import shutil
import threading
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder

In [1]:
# mounting google drive
from google.colab import drive
drive.mount('drive')

Mounted at drive


# Data Collection

## NCVS

function for retrieving NCVS personal victimization and personal population data

In [3]:
def NCVS_data():
    q1_url = 'https://data.ojp.usdoj.gov/resource/gcuy-rt5g.json?$limit=1000000'
    q2_url = 'https://data.ojp.usdoj.gov/resource/r4j4-fdwx.json?$limit=1000000'   
    response1 = requests.get(q1_url)
    response2 = requests.get(q2_url)
    pcv = response1.json()  #personal victimization
    pp = response2.json()   #personal population
    print(q1_url, response1)
    print(q2_url, response2)
    return pcv,pp

In [4]:
pcv,pp = NCVS_data()

https://data.ojp.usdoj.gov/resource/gcuy-rt5g.json?$limit=1000000 <Response [200]>
https://data.ojp.usdoj.gov/resource/r4j4-fdwx.json?$limit=1000000 <Response [200]>


Saving Personal Victimization data to JSON in google drive

In [5]:
with open("ncvs_pcv.json", "w") as outfile:
    json.dump(pcv, outfile)

In [9]:
colab_link = "/content/ncvs_pcv.json"
gdrive_link = "/content/drive/MyDrive/crime_stats_project/ncvs/ncvs_pcv.json"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/crime_stats_project/ncvs/ncvs_pcv.json'

Saving Personal Population data to JSON in google drive

In [11]:
with open("ncvs_pp.json", "w") as outfile:
    json.dump(pp, outfile)

In [12]:
colab_link = "/content/ncvs_pp.json"
gdrive_link = "/content/drive/MyDrive/crime_stats_project/ncvs/ncvs_pp.json"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/crime_stats_project/ncvs/ncvs_pp.json'

## NIBRS Reported offense count

In [13]:
# requesting the API for state names and abbreviation
url_abbr = "https://api.usa.gov/crime/fbi/sapi/api/agencies?API_KEY=pX3LYY2A6ahE5xVPUFqT4j5zq8jy7BflrVcOHJwx"
abbr_resp = requests.get(url_abbr)
api_dict = abbr_resp.json()

# looping over all states to get their corresponding abbreviation strings
name_abbr_dict = []
abbr_list = []
for state_ind in range(len(api_dict)):
  state_abbr_dict = {}
  state_abbr = list(api_dict.keys())[state_ind]
  state = list(api_dict[state_abbr].values())[0]
  state_name = state['state_name']
  state_abbr_dict['state_name'] = state_name
  state_abbr_dict['state_abbr'] = state_abbr
  # a list of dictionaries for all states and their abbreviations
  name_abbr_dict.append(state_abbr_dict)
  # a list of state abbreviations
  abbr_list.append(state_abbr)

In [15]:
# a list of all offeneses taken from the cde API documentation
offense_list = ['aggravated-assault','all-other-larceny','all-other-offenses','animal-cruelty','arson','assisting-or-promoting-prostitution',\
                'bad-checks','betting','bribery','burglary-breaking-and-entering','counterfeiting-forgery','credit-card-automated-teller-machine-fraud',\
                'destruction-damage-vandalism-of-property','driving-under-the-influence','drug-equipment-violations','drug-violations','drunkenness',\
                'embezzlement','extortion-blackmail','false-pretenses-swindle-confidence-game','fondling','gambling-equipment-violation',\
                'hacking-computer-invasion','human-trafficking-commerical-sex-acts','human-trafficking-commerical-involuntary-servitude','identity-theft',\
                'impersonation','incest','intimidation','justifiable-homicide','kidnapping-abduction','motor-vehicle-theft','murder-and-nonnegligent-manslaughter',\
                'negligent-manslaughter','operating-promoting-assiting-gambling','curfew-loitering-vagrancy-violations','peeping-tom','pocket-picking',\
                'pornography-obscence-material','prostitution','purchasing-prostitution','purse-snatching','rape','robbery','sexual-assult-with-an-object',\
                'sex-offenses-non-forcible','shoplifting','simple-assault','sodomy','sports-tampering','statutory-rape','stolen-property-offenses',\
                'theft-from-building','theft-from-coin-operated-machine-or-device','theft-from-motor-vehicle','theft-of-motor-vehicle-parts-or-accessories',\
                'theft-from-motor-vehicle','weapon-law-violation','welfare-fraud','wire-fraud','not-specified','liquor-law-violations']

# a list of offense categories based on cde API documentation
offenses_collected = ['crime-against-person', 'crime-against-property','crime-against-society','assault-offenses','homicide-offenses',\
                      'human-trafficking-offenses','sex-offenses','sex-offenses-non-forcible','fraud-offenses','larceny-theft-offenses',\
                      'drugs-narcotic-offenses','gambling-offenses','prostitution-offenses','all-offenses']

# a manually curated list of dictionaries for grouping single offenses based on the NIBRS
grouped_offense_list = [{'offense_group': 'animal-cruelty', 'single_offenses': ['animal-cruelty']},
                        {'offense_group': 'arson', 'single_offenses': ['arson']},
                        {'offense_group': 'assault-offenses', 'single_offenses': ['aggravated-assault','simple-assault','intimidation']},
                        {'offense_group': 'bribery', 'single_offenses': ['bribery']},
                        {'offense_group': 'burglary-breaking-and-entering' , 'single_offenses': ['burglary-breaking-and-entering']},
                        {'offense_group': 'embezzlement', 'single_offenses': ['embezzlement']},
                        {'offense_group': 'extortion-blackmail', 'single_offenses': ['extortion-blackmail']},
                        {'offense_group': 'fraud-offenses', 'single_offenses': ['false-pretenses-swindle-confidence-game','credit-card-automated-teller-machine-fraud','impersonation','welfare-fraud','wire-fraud','identity-theft','hacking-computer-invasion']},
                        {'offense_group': 'gambling-offenses', 'single_offenses': ['betting','operating-promoting-assiting-gambling','gambling-equipment-violation','sports-tampering']},
                        {'offense_group': 'homicide-offenses', 'single_offenses': ['murder-and-nonnegligent-manslaughter','negligent-manslaughter','justifiable-homicide']},
                        {'offense_group': 'human-trafficking', 'single_offenses': ['human-trafficking-commerical-sex-acts','human-trafficking-commerical-involuntary-servitude']},
                        {'offense_group': 'kidnapping-abduction', 'single_offenses': ['kidnapping-abduction']},
                        {'offense_group': 'larceny-theft-offenses', 'single_offenses': ['pocket-picking','purse-snatching','shoplifting','theft-from-building','theft-from-coin-operated-machine-or-device','theft-from-motor-vehicle','theft-of-motor-vehicle-parts-or-accessories','all-other-larceny' ]},
                        {'offense_group': 'motor-vehicle-theft', 'single_offenses': ['motor-vehicle-theft']},
                        {'offense_group': 'pornography-obscence-material', 'single_offenses': ['pornography-obscence-material']},
                        {'offense_group': 'prostitution-offenses', 'single_offenses': ['prostitution','assisting-or-promoting-prostitution','purchasing-prostitution']},
                        {'offense_group': 'robbery', 'single_offenses': ['robbery']},
                        {'offense_group': 'destruction-damage-vandalism-of-property' , 'single_offenses': ['destruction-damage-vandalism-of-property']},
                        {'offense_group': 'counterfeiting-forgery', 'single_offenses': ['counterfeiting-forgery']},
                        {'offense_group': 'sex-offenses', 'single_offenses': ['rape','sexual-assult-with-an-object','sodomy','statutory-rape','fondling','incest','sex-offenses-non-forcible']},
                        {'offense_group': 'stolen-property-offenses', 'single_offenses': ['stolen-property-offenses']},
                        {'offense_group': 'weapon-law-violation', 'single_offenses': ['weapon-law-violation']},
                        {'offense_group': 'drug-narcotic-offenses', 'single_offenses': ['drug-equipment-violations','drug-violations']},
                        {'offense_group': 'others', 'single_offenses': ['all-other-offenses','bad-checks','driving-under-the-influence','drunkenness','curfew-loitering-vagrancy-violations','peeping-tom','not-specified','liquor-law-violations']}]  
print('Number of offense categories = ', len(grouped_offense_list))

Number of offense categories =  24


In [16]:
# collecting the NIBRS crime data using multithreading operations

# a list of tuples to group urls and corresponding arugments
api_args = []
# a list of dictionaries to store the results of the API calls
offense_count_list = []

# grouping api urls along with offense group and state name and abbr into a list of tuples
for grouped_offenses in grouped_offense_list:
  offense_group = grouped_offenses['offense_group']
  for offense in grouped_offenses['single_offenses']:
      for state_info in name_abbr_dict:
        state_name = state_info['state_name']
        state_abbr = state_info['state_abbr']
        url = "https://api.usa.gov/crime/fbi/sapi/api/data/nibrs/" + offense + "/offender/states/" + state_abbr + "/COUNT?API_KEY=pX3LYY2A6ahE5xVPUFqT4j5zq8jy7BflrVcOHJwx" 
        args = (url, offense_group, state_name, state_abbr)
        api_args.append(args)


# an API call function to be processed in parallel with multithreaded operations
# the result of each API call is a list of dictionaries which is appeneded to the main offense count list
# offenses under the same group are assigned the offense group
def call_nibrs_api(api_args):
  time.sleep(1)
  resp = requests.get(api_args[0])
  resp_dict = resp.json()['results']
  for d in resp_dict:
    d['offense'] = api_args[1]
    d['state_name'] = api_args[2]
    d['state_abbr'] = api_args[3]
  offense_count_list.extend(resp_dict)
  return 

# define a multithreading operation with 12 workers
executor = ThreadPoolExecutor(12)
futures = []

# loop through the list of urls and their relative arguments and submit api calls in parallel
for api_call in api_args:
  future = executor.submit(call_nibrs_api, (api_call))
  futures.append(future)



In [24]:
print('Number of records = ', len(offense_count_list))

Number of records =  39650


##Recidivism data for the state of Georgia

Function for retrieving NJI data

In [25]:
def NIJ_data():
    q1_url = 'https://data.ojp.usdoj.gov/resource/ynf5-u8nk.json?$limit=1000000'
    response1 = requests.get(q1_url)
    data1 = response1.json()
    print(q1_url, response1)
    return data1

In [26]:
sn = NIJ_data()

https://data.ojp.usdoj.gov/resource/ynf5-u8nk.json?$limit=1000000 <Response [200]>


Saving NJI data to JSON in google drive

In [27]:
with open("nji.json", "w") as outfile:
    json.dump(sn, outfile)

In [28]:
colab_link = "/content/nji.json"
gdrive_link = "/content/drive/MyDrive/crime_stats_project/nji/nji_georgia.json"
shutil.copy(colab_link, gdrive_link)

'/content/drive/MyDrive/crime_stats_project/nji/nji_georgia.json'

## Firearm laws per state data

In [29]:
# loading the data from Gdrive
df_firearm = pd.read_excel('drive/MyDrive/crime_stats_project/firearm/DATABASE_0.xlsx')
df_firearm.head()

,state,year,felony,invcommitment,invoutpatient,danger,drugmisdemeanor,alctreatment,alcoholism,relinquishment,...,expartedating,dvrosurrender,dvrosurrendernoconditions,dvrosurrenderdating,expartesurrender,expartesurrendernoconditions,expartesurrenderdating,dvroremoval,stalking,lawtotal
0,Alabama,1991,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,15
1,Alabama,1992,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
2,Alabama,1993,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
3,Alabama,1994,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16
4,Alabama,1995,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,16


# Data Cleaning

##NCVS

In [30]:
pcvdf = pd.read_json("/content/drive/MyDrive/crime_stats_project/ncvs/ncvs_pcv.json")
ppdf = pd.read_json("/content/drive/MyDrive/crime_stats_project/ncvs/ncvs_pp.json")

In [31]:
pcvdf2 = pcvdf.drop(pcvdf[pcvdf.columns.difference(['idper','year','newoff','ager','race','sex','hincome1','educatn1','direl','notify','treatment','serious','offenderage','offendersex','offtracenew'])],axis = 1)
ppdf2 = ppdf.drop(ppdf[ppdf.columns.difference(['idper','year','ager','race','sex','hincome1','educatn1','citizen'])],axis = 1)

Dictionaries for Mapping Categorical Variables

In [32]:
crime_dict =  {1 : 'Sexual_Assault', 2 : 'Robbery', 3: 'Aggravated_Assault', 4 : 'Simple_Assault', 5: 'Larceny'}
race_dict = {1 : 'White', 2 : 'Black', 3: 'American_Indian/Alaska_Native', 4 : 'Asian/Native_Hawaiian', 5: 'Mixed'}
sex_dict = {1:'Male',2:'Female'}
age_dict = {1 : '12-17', 2 : '18-24', 3: '25-34', 4 : '35-49', 5: '50-64', 6: '65 or older'}
relation_dict = {1 : 'intimates', 2 : 'other_relatives', 3: 'acquaintances', 4 : 'strangers', 5: 'unknown', 6: 'unknown_num_of_offenders'}
offender_age_dict = {1: '11_or_younger',2 : '12-17', 3 : '18-29', 4: '30_or_older', 5 : 'grouo_of_diff_ages', 88: 'Residue'}
income_dict = {1 : '< $7500', 2 : '$7500 - $14999', 3: '$15000 - $24999', 4 : '$25000 - $34999',5:'$35000 - $49999',6: '$50000 - $74999', 7: '>= $75000', 88: 'unknown'}
edu_dict = {1 : 'No_Schooling', 2 : 'Grade_School', 3: 'Middle_School', 4 : 'High_School', 5: 'College', 88: 'Residue'}
reported_dict = {1 : 'Yes', 2 : 'No', 3: 'Unknown', 8 : 'Residue'}
treatment_dict = {0:'not_injured', 1 : 'not_treated', 2 : 'treated', 3: 'Unknown', 8 : 'Residue'}
injury_dict = {1 : 'no_injury', 2 : 'serious_injury', 3: 'minor_injury', 4 : 'rape_wo_other_injuries', 88: 'Residue'}
offender_sex_dict = {1:'Male',2:'Female',3:'Both',4:'Unknown',88:'Residue'}
offender_race_dict = {1 : 'White', 2 : 'Black', 3: 'American_Indian/Alaska_Native', 4 : 'Asian/Native_Hawaiian', 5: 'Mixed',6: 'Hispanic',7: 'Unknown',10: 'Mixed_Group',11: 'Unknown_Group',-1:'Invalid_until_2012Q1'}


Mapping category codes to categories

In [33]:
pcvdf2 = pcvdf2.replace({"newoff": crime_dict,'ager':age_dict,'race':race_dict,'sex':sex_dict,'hincome1':income_dict,'educatn1':edu_dict,'notify':reported_dict,'treatment':treatment_dict,'serious':injury_dict,'offenderage':offender_age_dict,'offendersex':offender_sex_dict,'offtracenew':offender_race_dict,'direl':relation_dict})
ppdf2 = ppdf2.replace({'race':race_dict,'ager':age_dict,'sex':sex_dict,'hincome1':income_dict,'educatn1':edu_dict})

Removing Entries with "Residue" values

In [34]:
pcvdf_filtered = pcvdf2[(pcvdf2['serious'] != 'Residue') & (pcvdf2['treatment'] !='Residue') & (pcvdf2['notify'] != 'Residue') & (pcvdf2['offendersex']!='Residue') & (pcvdf2['educatn1'] != 'Residue') & (pcvdf2['offenderage'] != 'Residue')]
ppdf_filtered = ppdf2[(ppdf2['educatn1'] != 'Residue')]

Renaming Columns

In [35]:
pcvdf_filtered.rename(columns = {'newoff':'crime','ager':'age','idper':'id','hincome1':'house_income','educatn1':'education_level','notify':'is_reported','treatment':'is_treated','serious':'injury_level','offtracenew':'offender_race','direl':'victim_offender_relation'}, inplace = True)
ppdf_filtered.rename(columns = {'idper':'id','ager':'age','hincome1':'house_income','educatn1':'education_level'}, inplace = True)

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [38]:
# saving the cleaned datasets to gdrive
pcvdf_filtered.to_csv('pcvdf_clean.csv', index = False)
!cp pcvdf_clean.csv "drive/MyDrive/crime_stats_project/ncvs"

ppdf_filtered.to_csv('ppdf_clean.csv', index = False)
!cp ppdf_clean.csv "drive/MyDrive/crime_stats_project/ncvs"

## NIBRS

In [39]:
# a dataframe to store the results of API data collection
df_nibrs = pd.DataFrame.from_dict(offense_count_list)
df_nibrs.rename(columns = {'data_year':'year'}, inplace = True)
df_nibrs.head()

,count,year,offense,state_name,state_abbr
0,54,2018,animal-cruelty,Hawaii,HI
1,58,2019,animal-cruelty,Hawaii,HI
2,48,2020,animal-cruelty,Hawaii,HI
3,55,2021,animal-cruelty,Hawaii,HI
4,1,2013,animal-cruelty,Michigan,MI


In [40]:
# summing the offense counts of offenses under the same offense group
# for example aggravated-assault, simple-assault, intimidation appear as different entries
# but under the same offense type (assault-offenses) so their counts are summed for each state and year combination

aggregation_functions = {'count': 'sum', 'year': 'first', 'offense': 'first', 'state_name': 'first', 'state_abbr': 'first'}
df_nibrs = df_nibrs.groupby(['year', 'offense', 'state_name','state_abbr'], as_index = False).aggregate(aggregation_functions).reindex(columns=df_nibrs.columns)

In [41]:
df_nibrs.head()

,count,year,offense,state_name,state_abbr
0,163,1991,arson,Alabama,AL
1,107,1991,arson,North Dakota,ND
2,1248,1991,arson,South Carolina,SC
3,70130,1991,assault-offenses,Alabama,AL
4,9,1991,assault-offenses,Illinois,IL


In [42]:
df_nibrs['offense'].value_counts()

sex-offenses                                904
assault-offenses                            904
larceny-theft-offenses                      900
drug-narcotic-offenses                      894
fraud-offenses                              891
burglary-breaking-and-entering              890
destruction-damage-vandalism-of-property    889
counterfeiting-forgery                      885
stolen-property-offenses                    884
motor-vehicle-theft                         884
weapon-law-violation                        883
robbery                                     880
kidnapping-abduction                        872
arson                                       859
embezzlement                                853
homicide-offenses                           844
pornography-obscence-material               834
prostitution-offenses                       827
extortion-blackmail                         806
gambling-offenses                           686
bribery                                 

In [43]:
# rearranging the columns for more clarity
cols_nibrs = ['year', 'state_name', 'state_abbr', 'offense', 'count']
df_nibrs = df_nibrs[cols_nibrs]
# saving the clean data on Gdrive for further access
df_nibrs.to_csv('nibrs_offense_count.csv', index = False)
!cp nibrs_offense_count.csv "drive/MyDrive/crime_stats_project/nibrs"

##Recidivism data for the state of Georgia

In [44]:
njidf = pd.read_json("/content/drive/MyDrive/crime_stats_project/nji/nji_georgia.json")

Creating a column name map for clearer column titles

In [45]:
rename_dict = {
    'supervision_risk_score_first':'first_parole_risk_score',
    'supervision_level_first':'first_parole_supervision_level',
    'prior_arrest_episodes_felony':'prior_felony_arrests',
    'prior_arrest_episodes_misd':'prior_misdemeanor_arrests',
    'prior_arrest_episodes_violent':'prior_violent_arrests',
    'prior_arrest_episodes_property':'prior_property_arrests',
    'prior_arrest_episodes_drug':'prior_drug_arrests',
    'prior_arrest_episodes':'prior_parole_violation_arrests',
    'prior_arrest_episodes_1':'prior_domestic_violence_arrests',
    'prior_arrest_episodes_2':'prior_gun_charges_arrests',
    'prior_conviction_episodes': 'prior_felony_convictions',
    'prior_conviction_episodes_1': 'prior_misdemeanor_convictions',
    'prior_conviction_episodes_2': 'prior_violent_convictions',
    'prior_conviction_episodes_3': 'prior_property_convictions',
    'prior_conviction_episodes_4': 'prior_drug_convictions',
    'prior_conviction_episodes_5': 'prior_parole_violation_convictions',
    'prior_conviction_episodes_6': 'prior_domestic_violence_convictions',
    'prior_conviction_episodes_7': 'prior_gun_charges_convictions',
    'prior_revocations_parole': 'prior_revoked_paroles',
    'prior_revocations_parole': 'prior_revoked_probations',
    'condition_mh_sa': 'parole_releases_for_mental_health',
    'condition_cog_ed': 'parole_releases_for_congitive_skills',
    'condition_other': 'parole_releases_other',
    'violations': 'parole_violations_monitoring',
    'violations_instruction': 'parole_violations_instructions',
    'violations_failtoreport': 'parole_violations_fail_to_Report',
    'delinquency_reports': 'parole_delinquency_reports',
    'program_attendances': 'parole_program_attendances',
    'program_unexcusedabsences': 'parole_program_unexcusedabsences',
    'residence_changes': 'parole_residence_changes'
}

Changing Column titles

In [46]:
njidf.rename(columns = rename_dict, inplace = True)

Checking for Null values

In [47]:
njidf.isnull().sum()

id                                         0
gender                                     0
race                                       0
age_at_release                             0
residence_puma                             0
gang_affiliated                         3167
first_parole_risk_score                  475
first_parole_supervision_level          1720
education_level                            0
dependents                                 0
prison_offense                          3277
prison_years                               0
prior_felony_arrests                       0
prior_misdemeanor_arrests                  0
prior_violent_arrests                      0
prior_property_arrests                     0
prior_drug_arrests                         0
prior_parole_violation_arrests             0
prior_domestic_violence_arrests            0
prior_gun_charges_arrests                  0
prior_felony_convictions                   0
prior_misdemeanor_convictions              0
prior_viol

The presence of null values in this context is not meaningless.Therefore, We will choose to keep records containing them

In [48]:
# saving the clean data on Gdrive for further access
njidf.to_csv('njidf_clean.csv', index = False)
!cp njidf_clean.csv "drive/MyDrive/crime_stats_project/nji"

## Firearm dataset

In [49]:
# in the firearm dataset there are almost 150 laws for gun regulations
# to reduce the number of columns we will replace each law variable with its category and sum all laws applied within each category
# using this mechanism we still get to know how much is gun controlled in each state without diving into each law's minor details
dealer_regulations = ['dealer', 'dealerh', 'recordsdealer', 'recordsdealerh', 'recordsall', 'recordsallh', 'reportdealer', 'reportdealerh', \
                      'reportall', 'reportallh', 'purge', 'residential', 'theft', 'security', 'inspection', 'liability', 'junkgun']
buyer_regulations = ['waiting', 'waitingh', 'permit', 'permith', 'permitlaw', 'fingerprint', 'training', 'registration', 'registrationh', \
                     'defactoreg', 'defactoregh', 'age21handgunsale', 'age18longgunsale', 'age21longgunsaled', 'age21longgunsale', 'loststolen', 'onepermonth']
prohibitions_for_highrisk_gun_possession = ['felony', 'violent', 'violenth', 'violentpartial', 'invcommitment', 'invoutpatient', 'danger',\
                                            'drugmisdemeanor', 'alctreatment', 'alcoholism', 'relinquishment']
background_checks = ['universal', 'universalh', 'gunshow', 'gunshowh', 'universalpermit', 'universalpermith', 'backgroundpurge', \
                     'threedaylimit', 'mentalhealth', 'statechecks', 'statechecksh']
ammunition_regulations = ['ammlicense', 'ammrecords', 'ammpermit', 'ammrestrict', 'amm18', 'amm21h', 'ammbackground']
possession_regulations = ['age21handgunpossess', 'age18longgunpossess', 'age21longgunpossess', 'gvro', 'gvrolawenforcement', \
                          'college', 'collegeconcealed', 'elementary', 'opencarryh', 'opencarryl', 'opencarrypermith', 'opencarrypermitl']
concealed_carry_permitting = ['permitconcealed', 'mayissue', 'showing', 'ccrevoke', 'ccbackground', 'ccbackgroundnics', 'ccrenewbackground']
assault_weapons_and_large_capacity_magazines = ['assault', 'onefeature', 'assaultlist', 'assaultregister', 'assaulttransfer', 'magazine', 'tenroundlimit', 'magazinepreowned']
child_access_prevention = ['lockd', 'lockp', 'lockstandards', 'locked', 'capliability', 'capaccess', 'capuses', 'capunloaded', 'cap18', 'cap16', 'cap14']
gun_trafficking = ['traffickingbackground', 'traffickingprohibited', 'traffickingprohibitedh', 'strawpurchase', 'strawpurchaseh', 'microstamp', 'personalized']
domestic_violence = ['mcdv', 'mcdvdating', 'mcdvsurrender', 'mcdvsurrendernoconditions', 'mcdvsurrenderdating', 'mcdvremovalallowed', 'mcdvremovalrequired', \
                     'incidentremoval', 'incidentall', 'dvro', 'dvrodating', 'exparte', 'expartedating', 'dvrosurrender', 'dvrosurrendernoconditions', \
                     'dvrosurrenderdating', 'expartesurrender', 'expartesurrendernoconditions', 'expartesurrenderdating', 'dvroremoval', 'stalking']
preemption = ['preemption','preemptionnarrow','preemptionbroad']
immunity = ['immunity']
stand_your_ground = ['nosyg']

# summing the laws applied within each category and add a column for the category
df_firearm['dealer_regulations']= df_firearm[dealer_regulations].sum(axis=1)
df_firearm['buyer_regulations']= df_firearm[buyer_regulations].sum(axis=1)
df_firearm['prohibitions_for_highrisk_gun_possession']= df_firearm[prohibitions_for_highrisk_gun_possession].sum(axis=1)
df_firearm['background_checks']= df_firearm[background_checks].sum(axis=1)
df_firearm['ammunition_regulations']= df_firearm[ammunition_regulations].sum(axis=1)
df_firearm['possession_regulations']= df_firearm[possession_regulations].sum(axis=1)
df_firearm['concealed_carry_permitting']= df_firearm[concealed_carry_permitting].sum(axis=1)
df_firearm['assault_weapons_and_large_capacity_magazines']= df_firearm[assault_weapons_and_large_capacity_magazines].sum(axis=1)
df_firearm['child_access_prevention']= df_firearm[child_access_prevention].sum(axis=1)
df_firearm['gun_trafficking']= df_firearm[gun_trafficking].sum(axis=1)
df_firearm['domestic_violence']= df_firearm[domestic_violence].sum(axis=1)
df_firearm['preemption']= df_firearm[preemption].sum(axis=1)
df_firearm['stand_your_ground']= df_firearm[stand_your_ground].sum(axis=1)

# selecting only the columns with the law categories

df_firearm = df_firearm[['state','year','dealer_regulations', 'buyer_regulations', 'prohibitions_for_highrisk_gun_possession', 'background_checks',\
                         'ammunition_regulations', 'ammunition_regulations', 'possession_regulations', 'concealed_carry_permitting',\
                         'assault_weapons_and_large_capacity_magazines', 'child_access_prevention', 'gun_trafficking', 'domestic_violence',\
                         'preemption', 'stand_your_ground', 'immunity']]

df_firearm.head()

,state,year,dealer_regulations,buyer_regulations,prohibitions_for_highrisk_gun_possession,background_checks,ammunition_regulations,ammunition_regulations,possession_regulations,concealed_carry_permitting,assault_weapons_and_large_capacity_magazines,child_access_prevention,gun_trafficking,domestic_violence,preemption,stand_your_ground,immunity
0,Alabama,1991,7,1,1,0,0,0,0,3,0,0,0,0,1,1,1
1,Alabama,1992,7,1,1,0,0,0,0,3,0,0,0,1,1,1,1
2,Alabama,1993,7,1,1,0,0,0,0,3,0,0,0,1,1,1,1
3,Alabama,1994,7,1,1,0,0,0,0,3,0,0,0,1,1,1,1
4,Alabama,1995,7,1,1,0,0,0,0,3,0,0,0,1,1,1,1


In [50]:
# saving the cleaned dataset to gdrive
df_firearm.to_csv('firearm_laws.csv', index = False)
!cp firearm_laws.csv "drive/MyDrive/crime_stats_project/firearm"